In [88]:
#Libraries
#Libraries
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import joblib

In [108]:
#Import fit File

fit_file = 'fit_file_test.csv'

df = pd.read_csv(fit_file, index_col=False, parse_dates=['tiemstamp_AEST'])

df = df[['tiemstamp_AEST', 'distance', 'ascent', 'enhanced_speed',
              'heart_rate', 'cadence', 'power', 'userID']]

df.sort_values(by=['userID', 'tiemstamp_AEST'], inplace = True)

df = df.drop(columns=['userID'])
df['SessionID'] = 1

#Averages

df_avg = df.groupby('SessionID').mean()

#Maxes
df_max = df.groupby('SessionID').max()

#Session Length
df_length = df.groupby('SessionID').max()-df.groupby('SessionID').min()

#Lock required features
df_avg = df_avg[['enhanced_speed','heart_rate', 'cadence', 'power']]
df_max = df_max[['distance','ascent']]
df_length = df_length[['tiemstamp_AEST']]


#Convert time to session length in minutes
for i in range(1, len(df_length['tiemstamp_AEST'])+1):
    df_length['tiemstamp_AEST'][i] = (df_length['tiemstamp_AEST'][i].total_seconds())/60

df_length = df_length.rename(columns={"tiemstamp_AEST": "length"})
df_length = df_length.astype(float)

#Combine feature types
df_comb = pd.concat([df_avg, df_max, df_length], axis=1)
df_comb = round(df_comb,2)

clusterer = joblib.load('clustering_model.sav')
scaler = joblib.load('clustering_scaler.sav')

st_df = scaler.transform(df_comb)

cluster_label = clusterer.predict(st_df)
print("Fitfile cluster allocation:", format(cluster_label[0]))


Fitfile cluster allocation: 0
